In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("E:/python/testproject/20180621_LLM_with_GPS_train.xlsx")
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('(', '') for c in df.columns]
df.columns = [c.replace(')', '') for c in df.columns]

In [38]:
df_droped = df[np.logical_not(pd.isna(df["completed_time"]))]
df_droped = df_droped[np.logical_not(pd.isna(df_droped["driver_taken_time"]))]
df_droped = df_droped[np.logical_not(pd.isna(df_droped["pickedup_time"]))]
#df_droped = df[df.order_type == "Immediate"]
#df_droped = df_droped[df_droped.order_status == "Completed"]
#df_droped = df_droped[df_droped.service_type == "VAN"]

countcomma = lambda x: x.count(",") - 1
df_droped["pathstops"] = list(map(countcomma, df_droped.path.astype("str")))
#df_droped = df_droped[df_droped["pathcomma"] == 1]

df_droped["path_from"] = df_droped.path.str.split(",", 1, expand=True).iloc[:,0]
df_droped["path_to"] = df_droped.path.str.rsplit(",", 1, expand=True).iloc[:,1]

#df_path = df_droped.path.str.split(",", expand=True)
#df_path.columns = ["path_from","path_to"]
#df_droped = pd.concat([df_droped, df_path], axis = 1)
# sum(pd.isna(df_droped["completed_time"]) + 0)
# still two missing, will drop

#df_droped["tips"] = df_droped["total_price_+_tips"] - df_droped["total_price"]
#df_droped["total_adj"] = df_droped["transaction_value"] - df_droped["total_price"]

df_droped["placed_year"] = df_droped["placed_time"].dt.year
df_droped["placed_month"] = df_droped["placed_time"].dt.month
df_droped["placed_weekday"] = df_droped["placed_time"].dt.weekday
df_droped["placed_hour"] = df_droped["placed_time"].dt.hour
df_droped["placed_minute"] = df_droped["placed_time"].dt.minute

df_droped["taken_minutes"] = (df_droped["driver_taken_time"] - df_droped["placed_time"]).astype('timedelta64[m]')
df_droped["pickup_minutes"] = (df_droped["pickedup_time"] - df_droped["placed_time"]).astype('timedelta64[m]')

#df_droped["latitude_dis"] = abs(df_droped["driver_latitude_pick_up"] - df_droped["driver_latitude_complete"])
#df_droped["longitude_dis"] = abs(df_droped["driver_longitude_pick_up"] - df_droped["driver_longitude_complete"])

#columns_to_cut = ["driver_latitude_pick_up", "driver_longitude_pick_up", "driver_latitude_complete", "driver_longitude_complete"]

#for i in columns_to_cut:
#    df_droped[i] = pd.qcut(df_droped[i],50)
    
df_droped["placed_minute"] = pd.qcut(df_droped["placed_minute"],6)
    
#impute na with 0 or mean for some continues
#df_droped[["subsidy", "adjustment"]] = df_droped[["subsidy", "adjustment"]].fillna(0)
#df_droped["dis"] = np.sqrt(df_droped["latitude_dis"]**2 + df_droped["longitude_dis"]**2)
#df_droped.loc[df_droped["dis"] >1, "dis"] = np.nan
#df_droped["dis"] = df_droped["dis"].fillna(np.mean(df_droped["dis"]))

df_droped["taken_minutes"] = df_droped["taken_minutes"].fillna(np.mean(df_droped["taken_minutes"]))
df_droped["pickup_minutes"] = df_droped["pickup_minutes"].fillna(np.mean(df_droped["pickup_minutes"]))

df_droped["target"] = (df_droped.completed_time - df_droped.placed_time)/np.timedelta64(1,'h') > 1

df_droped = df_droped.drop(["order_status", "order_date_time", "cs_comment", "cancel_type", "cancel_reason", "cancelled_by",
              "cancelled_time", "is_responded", "reject_time", "arrival_time_geofence",
              "driver_latitude_take_order", "driver_longitude_take_order", "path", 
              "completed_time", "placed_time","total_price_+_tips"], axis = 1)

# test set

In [10]:
df_tmp = pd.read_excel("E:/python/testproject/LLM_test.xlsx")
df_tmp.columns = [c.replace(' ', '_') for c in df_tmp.columns]
df_tmp.columns = [c.replace('(', '') for c in df_tmp.columns]
df_tmp.columns = [c.replace(')', '') for c in df_tmp.columns]

In [11]:
df_test_droped = df_tmp.drop(["order_status"], axis = 1)
time_variables = ["placed_time", "driver_taken_time", "pickedup_time"]
for v in time_variables:
    df_test_droped[v] = pd.to_datetime(df_test_droped[v])
    
df_test_droped["special_request"] = df_test_droped["order_remarks"]
#df_test_droped = df[df.order_type == "Immediate"]
#df_test_droped = df_test_droped[df_test_droped.order_status == "Completed"]
#df_test_droped = df_test_droped[df_test_droped.service_type == "VAN"]

countcomma = lambda x: x.count(",") - 1
df_test_droped["pathstops"] = list(map(countcomma, df_test_droped.path.astype("str")))
#df_test_droped = df_test_droped[df_test_droped["pathcomma"] == 1]

df_test_droped["path_from"] = df_test_droped.path.str.split(",", 1, expand=True).iloc[:,0]
df_test_droped["path_to"] = df_test_droped.path.str.rsplit(",", 1, expand=True).iloc[:,1]

#df_path = df_test_droped.path.str.split(",", expand=True)
#df_path.columns = ["path_from","path_to"]
#df_test_droped = pd.concat([df_test_droped, df_path], axis = 1)
# sum(pd.isna(df_test_droped["completed_time"]) + 0)
# still two missing, will drop

#df_test_droped["tips"] = df_test_droped["total_price_+_tips"] - df_test_droped["total_price"]
#df_test_droped["total_adj"] = df_test_droped["transaction_value"] - df_test_droped["total_price"]

df_test_droped["placed_year"] = df_test_droped["placed_time"].dt.year
df_test_droped["placed_month"] = df_test_droped["placed_time"].dt.month
df_test_droped["placed_weekday"] = df_test_droped["placed_time"].dt.weekday
df_test_droped["placed_hour"] = df_test_droped["placed_time"].dt.hour
df_test_droped["placed_minute"] = df_test_droped["placed_time"].dt.minute
df_test_droped["placed_minute"] = pd.qcut(df_test_droped["placed_minute"], 6)

df_test_droped["taken_minutes"] = (df_test_droped["driver_taken_time"] - df_test_droped["placed_time"]).astype('timedelta64[m]')
df_test_droped["pickup_minutes"] = (df_test_droped["pickedup_time"] - df_test_droped["placed_time"]).astype('timedelta64[m]')

df_test_droped["taken_minutes"] = df_test_droped["taken_minutes"].fillna(np.mean(df_test_droped["taken_minutes"]))
df_test_droped["pickup_minutes"] = df_test_droped["pickup_minutes"].fillna(np.mean(df_test_droped["pickup_minutes"]))

df_test_droped = df_tmp[df_tmp.order_type == "Immediate"]
df_test_droped = df_test_droped[df_test_droped.order_status == "Completed"]
df_test_droped = df_test_droped[df_test_droped.service_type == "VAN"]

countcomma = lambda x: x.count(",")
df_test_droped["pathcomma"] = list(map(countcomma, df_test_droped.path))
df_test_droped = df_test_droped[df_test_droped["pathcomma"] == 1]

df_path = df_test_droped.path.str.split(",", expand=True)
df_path.columns = ["path_from","path_to"]
df_test_droped = pd.concat([df_test_droped, df_path], axis = 1)

df_test_droped["tips"] = df_test_droped["total_price_+_tips"] - df_test_droped["total_price"]
df_test_droped["total_adj"] = df_test_droped["transaction_value"] - df_test_droped["total_price"]

df_test_droped["placed_year"] = df_test_droped["placed_time"].dt.year
df_test_droped["placed_month"] = df_test_droped["placed_time"].dt.month
df_test_droped["placed_weekday"] = df_test_droped["placed_time"].dt.weekday
df_test_droped["placed_hour"] = df_test_droped["placed_time"].dt.hour
df_test_droped["placed_minute"] = df_test_droped["placed_time"].dt.minute

df_test_droped["latitude_dis"] = abs(df_test_droped["driver_latitude_pick_up"] - df_test_droped["driver_latitude_complete"])
df_test_droped["longitude_dis"] = abs(df_test_droped["driver_longitude_pick_up"] - df_test_droped["driver_longitude_complete"])

columns_to_cut = ["driver_latitude_pick_up", "driver_longitude_pick_up", "driver_latitude_complete", "driver_longitude_complete"]

for i in columns_to_cut:
    df_test_droped[i] = pd.qcut(df_test_droped[i],50)
    
    
df_test_droped["placed_minute"] = pd.qcut(df_test_droped["placed_minute"], 6)
    
#impute na with 0 or mean for some continues
df_test_droped[["subsidy", "adjustment"]] = df_test_droped[["subsidy", "adjustment"]].fillna(0)
df_test_droped["dis"] = np.sqrt(df_test_droped["latitude_dis"]**2 + df_test_droped["longitude_dis"]**2)
#delete the outlier value
df_test_droped.loc[df_test_droped["dis"] >1, "dis"] = np.nan
df_test_droped["dis"] = df_test_droped["dis"].fillna(np.mean(df_test_droped["dis"]))

df_test_droped = df_test_droped.drop(["order_status", "service_type", "order_date_time", "cs_comment", "cancel_type", "cancel_reason",
              "cancelled_by", "cancelled_time", "is_responded", "reject_time", "arrival_time_geofence",
              "order_type", "driver_latitude_take_order", "driver_longitude_take_order", "pathcomma", "path", 
              "placed_time","total_price_+_tips", "latitude_dis", "longitude_dis"], axis = 1)

In [39]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#encode special_request
df_request = pd.DataFrame(data = {"request" :df_droped.special_request.str.cat(sep=',').split(",")})
unique_request = df_request.request.unique()

for i in unique_request:
    df_droped[i] = df_droped.special_request.str.contains(i).fillna(0)

df_droped[unique_request] = df_droped[unique_request].astype(int)

for i in unique_request:
    df_test_droped[i] = df_test_droped.special_request.str.contains(i).fillna(0)

df_test_droped[unique_request] = df_test_droped[unique_request].astype(int)

categorical_headers = ["final_subset", "payment_method", "corporate_code", "client_id", "promo_code",
                       "path_from", "path_to", "placed_year", "placed_month", "order_type", "service_type",
                       "placed_weekday", "placed_hour", "placed_minute"]
df_droped[categorical_headers] = df_droped[categorical_headers].astype(str)
df_test_droped[categorical_headers] = df_test_droped[categorical_headers].astype(str)

# define objects that can encode each variable as integer    
encoders = dict() 

# train all encoders (special case the target 'income')
for col in categorical_headers+["target"]:
    
    if col in ["target"]:
        tmp = LabelEncoder()
        df_droped[col] = tmp.fit_transform(df_droped[col])
    else:
        # integer encoded variables
        df_droped[col] = df_droped[col].str.strip()
        encoders[col] = LabelEncoder()
        df_droped[col + "_int"] = encoders[col].fit_transform(df_droped[col])
        
for col in categorical_headers:
        # integer encoded variables
        df_test_droped[col] = df_test_droped[col].str.strip()
        encoders[col] = LabelEncoder()
        df_test_droped[col + "_int"] = encoders[col].fit_transform(df_test_droped[col])

# scale the numeric, continuous variables
numeric_headers = ["transaction_value", "taken_minutes", "pickup_minutes"]


for col in numeric_headers:
    df_droped[col] = df_droped[col].astype(np.float)
    
    ss = StandardScaler()
    df_droped[col] = ss.fit_transform(df_droped[col].values.reshape(-1, 1))
    
for col in numeric_headers:
    df_test_droped[col] = df_test_droped[col].astype(np.float)
    
    ss = StandardScaler()
    df_test_droped[col] = ss.fit_transform(df_test_droped[col].values.reshape(-1, 1))
        
df_droped.head()

,order_id,final_subset,service_type,special_request,promo_code,payment_method,total_price,adjustment,transaction_value,corporate_code,...,promo_code_int,path_from_int,path_to_int,placed_year_int,placed_month_int,order_type_int,service_type_int,placed_weekday_int,placed_hour_int,placed_minute_int
0,1962533,A,TON55,TAILBOARD,nan,Cash,175,NaN,0.671535,EVCS,...,1081,56,36,1,4,2,3,4,3,1
2,1638808,A,VAN,SIXFT,nan,Cash,75,NaN,-0.667038,EVCS,...,1081,10,159,0,1,2,4,2,7,0
3,1784143,A,VAN,NaN,nan,Cash,75,NaN,-0.667038,nan,...,1081,120,49,0,3,2,4,5,5,2
4,1687581,A,VAN,NaN,nan,Cash,130,NaN,0.069177,EVCS,...,1081,8,144,0,2,2,4,2,6,1
5,1481932,A,VAN,NaN,nan,Cash,55,NaN,-0.934753,EVCS,...,1081,46,59,0,9,2,4,5,2,1


In [40]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

categorical_headers_ints = [x+'_int' for x in categorical_headers] + ["SIXFT", "PET", "NEWCAR", "TROLLEY3", "ONSITE",
                                                                      "OLD_CARS", "HOURLY", "ENGLISH", "STANDARD_MOVING",
                                                                      "VITO", "EXHIBIT"]
categorical_headers_ints = list(set(categorical_headers_ints).difference(set(["client_id_int", "promo_code_int", "corporate_code_int"])))


df_train, df_validation = train_test_split(df_droped, test_size=0.2)
df_test = df_test_droped

y_train = df_train["target"]
y_validation = df_validation["target"]

#Save off the numeric features
X_train_num =  df_train[numeric_headers].values
X_validation_num = df_validation[numeric_headers].values
X_test_num =  df_test[numeric_headers].values

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input, Dropout
from keras.layers import Embedding, Flatten, concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [50]:
cross_columns = [["path_from","path_to"],
                 #["driver_latitude_pick_up", "driver_longitude_pick_up"],
                 #["driver_latitude_complete", "driver_longitude_complete"],
                 ["placed_year", "placed_month"],
                 ["placed_month", "placed_weekday", "placed_hour"],
                 ["placed_weekday", "placed_hour", "placed_minute"],
                 ["order_type", "placed_hour","placed_weekday"],
                 ["order_type", "service_type"],
                 ["pathstops"],
                 ["client_id"],
                 ["promo_code"],
                 ["corporate_code"]]

embed_branches = []
X_ints_train = []
X_ints_validation = []
X_ints_test = []
all_inputs = []
all_wide_branch_outputs = []
all_deep_branch_outputs = []

for cols in cross_columns:
    # encode crossed columns as ints for the embedding
    enc = LabelEncoder()
    
    # create crossed labels
    if len(cols) > 1 :
        X_crossed_train = df_train[cols].apply(lambda x: "_".join(x), axis=1)
        X_crossed_validation = df_validation[cols].apply(lambda x: "_".join(x), axis=1)
        X_crossed_test = df_test[cols].apply(lambda x: "_".join(x), axis=1)
    else:
        X_crossed_train = df_train[cols].iloc[:,0].astype("str")
        X_crossed_validation = df_validation[cols].iloc[:,0].astype("str")
        X_crossed_test = df_test[cols].iloc[:,0].astype("str")
    
    enc.fit(np.hstack((X_crossed_train.values, X_crossed_validation.values, X_crossed_test.values)))
    X_crossed_train = enc.transform(X_crossed_train)
    X_crossed_validation = enc.transform(X_crossed_validation)
    X_crossed_test = enc.transform(X_crossed_test)
    X_ints_train.append( X_crossed_train )
    X_ints_validation.append( X_crossed_validation )
    X_ints_test.append( X_crossed_test )
    
    # get the number of categories
    N = max(X_ints_train[-1]+2)
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype="int32", name = "_".join(cols))
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, 
                  output_dim=int(np.log2(N)), 
                  input_length=1, name = "_".join(cols)+"_embed")(inputs)
    x = Flatten()(x)
    all_wide_branch_outputs.append(x)
        
# merge the branches together
wide_branch = concatenate(all_wide_branch_outputs, name="wide_concat")
wide_branch = Dropout(0.3)(wide_branch)
wide_branch = Dense(units=400,activation="relu",name="wide_combined_1")(wide_branch)
wide_branch = Dropout(0.3)(wide_branch)
#wide_branch = Dense(units=200,activation="relu",name="wide_combined_2")(wide_branch)
#wide_branch = Dropout(0.3)(wide_branch)
wide_branch = Dense(units=2,activation="sigmoid",name="wide_combined")(wide_branch)


# reset this input branch
all_deep_branch_outputs = []
# add in the embeddings
for col in categorical_headers_ints:
    # encode as ints for the embedding
    X_ints_train.append( df_train[col].values )
    X_ints_validation.append( df_validation[col].values )
    X_ints_test.append( df_test[col].values )
    
    # get the number of categories
    N = max(df_droped[col]) + 1
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype="int32", name=col)
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, 
                  output_dim=int(np.sqrt(N)), 
                  input_length=1, name=col+"_embed")(inputs)
    x = Flatten()(x)
    all_deep_branch_outputs.append(x)
    
# also get a dense branch of the numeric features
all_inputs.append(Input(shape=(X_train_num.shape[1],),
                        sparse=False,
                        name="numeric_data"))

x = all_inputs[-1]
#x = Dropout(0.15)(all_inputs[-1])
x = Dense(units=30, activation="relu",name="numeric_1")(x)

all_deep_branch_outputs.append( x )

# merge the deep branches together
deep_branch = concatenate(all_deep_branch_outputs,name="concat_embeds")
#deep_branch = Dropout(0.2)(deep_branch)
deep_branch = Dense(units=200,activation="relu", name="deep1")(deep_branch)
#deep_branch = Dropout(0.2)(deep_branch)
deep_branch = Dense(units=100,activation="relu", name="deep2")(deep_branch)
#deep_branch = Dropout(0.2)(deep_branch)
deep_branch = Dense(units=50,activation="relu", name="deep3")(deep_branch)
#deep_branch = Dropout(0.2)(deep_branch)
deep_branch = Dense(units=25,activation="relu", name="deep4")(deep_branch)
#deep_branch = Dropout(0.2)(deep_branch)
deep_branch = Dense(units=10,activation="relu", name="deep5")(deep_branch)
#deep_branch = Dropout(0.2)(deep_branch)
deep_branch = Dense(units=25,activation="relu", name="deep6")(deep_branch)
#eep_branch = Dropout(0.2)(deep_branch)
deep_branch = Dense(units=50,activation="relu", name="deep7")(deep_branch)
   
final_branch = concatenate([wide_branch, deep_branch],name="concat_deep_wide")
#final_branch = Dropout(0.2)(final_branch)
final_branch = Dense(units=100,activation="relu",name="combined_1")(final_branch)
#final_branch = Dropout(0.2)(final_branch)
#final_branch = Dense(units=50,activation="relu",name="combined_2")(final_branch)
#final_branch = Dropout(0.2)(final_branch)
final_branch = Dense(units=1,activation="sigmoid",name="combined_deep")(final_branch)

model = Model(inputs=all_inputs, outputs=final_branch)

In [349]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(model).create(prog='dot', format='svg'))

Exception: "dot.exe" not found in path.

In [51]:
%%time

model.compile(optimizer='Adadelta',
              loss='binary_crossentropy',
              metrics=['acc', 'mse'])

# lets also add the history variable to see how we are doing
# and lets add a validation set to keep track of our progress

history = model.fit(X_ints_train+ [X_train_num],
                    y_train, 
                    epochs=5, 
                    batch_size=256, 
                    verbose=1, 
                    validation_data = (X_ints_validation + [X_validation_num], y_validation),
                    callbacks=[EarlyStopping(monitor='val_loss', patience=2)])
#callbacks=[EarlyStopping(monitor='val_loss', patience=4)]

Train on 372397 samples, validate on 93100 samples
Epoch 1/5
372397/372397 [==============================] - 34s 92us/step - loss: 0.5112 - acc: 0.7527 - mean_squared_error: 0.1702 - val_loss: 0.4626 - val_acc: 0.7877 - val_mean_squared_error: 0.1506
Epoch 2/5
372397/372397 [==============================] - 30s 81us/step - loss: 0.4555 - acc: 0.7919 - mean_squared_error: 0.1482 - val_loss: 0.4297 - val_acc: 0.8083 - val_mean_squared_error: 0.1384
Epoch 3/5
372397/372397 [==============================] - 30s 81us/step - loss: 0.4334 - acc: 0.8050 - mean_squared_error: 0.1399 - val_loss: 0.4130 - val_acc: 0.8131 - val_mean_squared_error: 0.1324
Epoch 4/5
372397/372397 [==============================] - 30s 81us/step - loss: 0.4250 - acc: 0.8090 - mean_squared_error: 0.1369 - val_loss: 0.4041 - val_acc: 0.8237 - val_mean_squared_error: 0.1289
Epoch 5/5
372397/372397 [==============================] - 30s 81us/step - loss: 0.4176 - acc: 0.8132 - mean_squared_error: 0.1342 - val_loss: 0.

In [52]:
from sklearn import metrics as mt

y_prob =model.predict(X_ints_validation + [X_validation_num])

yhat = y_prob > 0.5
                
print(mt.confusion_matrix(y_validation,yhat), "\n", "acc:", mt.accuracy_score(y_validation,yhat), 
      "\n", "f1_score:", mt.f1_score(y_validation,yhat), "\n", "brier_score:", mt.brier_score_loss(y_validation,y_prob))

[[51294  5057]
 [11156 25593]] 
 acc: 0.8258539205155746 
 f1_score: 0.759447469547026 
 brier_score: 0.12515882217307114


In [53]:
y_test_prob =model.predict(X_ints_test + [X_test_num])

yhat_test = y_test_prob > 0.5

In [54]:
df_predict = pd.DataFrame()
df_predict["order_id"] = df_test["order_id"]
df_predict["y_prob"] = y_test_prob
df_predict["yhat"] = yhat_test

df_predict.to_csv("E:/python/testproject/test_withtime_60.csv")

ValueError: could not convert string to float: '\ufeff2716182'